# Desafio 1 - Criptoanálise com Shor

In [1]:
from aux_functions import decoder, intercept_transmission
import numpy as np
from qat.lang import qrout, H, CNOT, X, SWAP
from qat.lang.AQASM import Program
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import get_default_qpu

## Interceptando uma Mensagem

Recentemente, nossos serviços de inteligência interceptaram uma mensagem
codificada enviada por um membro do grupo hacker *Black Mesa*. A mensagem
foi identificada como uma peça vital para decifrarmos um futuro ataque que será
realizado pelo grupo. Ao que tudo indica, a mensagem está encriptada pelo
algoritmo RSA. Sem acesso à chave privada não é possível descobrir o conteúdo da
mensagem.

A chave pública do par RSA foi encontrada dentro de um fórum usado pelos membros
desse grupo. Entretanto, em circunstâncias normais não é possível descobrir a
chave privada tendo de informação apenas a chave pública. A prova de segurança
do RSA está baseada na dificuldade computacional do problema de fatoração de
números inteiros, um problema reconhecidamente difícil e que levaria dias em um
supercomputador para resolvê-lo. E ao final desse tempo a mensagem já teria
perdido qualquer valor estratégico.

In [2]:
ciphertext, public_key = intercept_transmission()

print(f'A mensagem criptografada interceptada é', ''.join(str(p) for p in ciphertext))
print(f'A chave pública usada para encriptar essa mensagem é {public_key}')

A mensagem criptografada interceptada é 9069057656127761576564680346803392390489125368904891294632665866811336219192335712665876021576562665872769760213571919231336216681266581294639048926658503353923468034680357656127761576566681
A chave pública usada para encriptar essa mensagem é (5, 21)


Por nossa sorte, não estamos em circunstâncias normais. Por meio do algoritmo
de Shor, um algoritmo quântico para a fatoração de números inteiros, é possível
quebrar a criptografia RSA. 

## Decifrando a mensagem

Implemente o algoritmo de Shor capaz de fatorar o número $N$ que faz parte da chave pública *(e, N)*.

Use os fatores primos $p$ e $q$ descobertos para calcular $\phi(N)$ e recuperar o expoente privado $d$, reconstruindo assim a chave privada *(d, p, q)*

In [3]:
e, N = public_key

In [4]:
print(f"Número a ser fatorado {N}")

# numero de qubits no primeiro registrador.
n = int(np.ceil(np.log2(N)))
#n = int(np.ceil(2*np.log2(N)))
# numero de qubits no segundo registrador.
m = int(np.ceil(np.log2(N)))
# numero total de qubits
num_qubits = n+m

print(f"{n} qubits no registrador de medida")
print(f"{m} qubits no segundo registrador")

Número a ser fatorado 21
5 qubits no registrador de medida
5 qubits no segundo registrador


In [5]:
# Escreva seu código abaixo
a_list = [a for a in range(2,N) if np.gcd(a,N) == 1]
print(a_list)

a = 13

[2, 4, 5, 8, 10, 11, 13, 16, 17, 19, 20]


In [6]:
@qrout
def initialize_qubits(total_qubits, size_first_reg):
    for i in range(size_first_reg):
        H(i)
    X(total_qubits-1)

# visualizando o circuito de inicialização
initialize_qubits(num_qubits, n).display()

In [7]:
@qrout
def modular_exponentiation(i):
    for _ in range(2**i):
        X(0)
        X(1)
        X(2)
        X(3)
        X(4)
        SWAP(0,3)
        SWAP(0,1)
        SWAP(1,2)
        SWAP(2,3)
        

# visualizando a exponenciação modular
modular_exponentiation(0).ctrl().display()

In [8]:
# criadndo o objeto shor com a classe Program
shor = Program()
# alocando qubits no programa shor
first_reg = shor.qalloc(n)
second_reg = shor.qalloc(m)

### Escreva seu código abaixo

# Aplique a rotina de inicialização
shor.apply(initialize_qubits(num_qubits, n), first_reg, second_reg)

# Aplique a Exponenciação Modular
for i in range(n):
    rout = modular_exponentiation(i).ctrl()
    shor.apply(rout,first_reg[-1-i], second_reg)

# Aplique a QFT no primeiro registrador
QFT(n)(first_reg) #QFT(numero de qubits)(qr[0], qr[1] ...)


# transformando de programa para circuito 
circuit = shor.to_circ()

# visualizando o circuito
# Obs: na interface web do jupyter é possível dar zoom na imagem e visualizar melhor o circuito
%qatdisplay circuit --svg

In [9]:
# submeta o circuito para job
# Repare que será realizada apenas uma medida no circuito 
job = circuit.to_job(nbshots=50, qubits=first_reg)
result = get_default_qpu().submit(job)

# Aqui imprimimos os estados medidos e suas respectivas probabilidades
# A lista states armazenará todos os estados medidos (ou apenas um unico estado se nbshots=1) para pos processamento
states = []
for sample in result:
     print("State %s: probability %s " % (sample.state, sample.probability))
     states.append(sample.state)

/home/omp/anaconda3/envs/QML/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


State |00000>: probability 0.56 
State |00001>: probability 0.44 


In [10]:
# O codigo abaixo converte os estados medidos no primeiro registrador em decimais
# Criamos uma lista string onde cada elemento corresponde a '0's ou '1's medidos no primeiro registrador 
decimals = set([int(''.join(state.value[0][::-1]),2) for state in states])
decimals

{0, 16}

In [11]:
from labmath import multord  # pip install labmath
# funcao que retorna o denominador de l/2^n
def fraction(num, denom):
    if denom==0:
        raise ZeroDivisionError("Denominador igual a zero")
    d = np.gcd(num, denom)
    num = num//d
    denom = denom//d
    return (num, denom)
# Encontrando os fatores primos de N
prime_factors = ()
success = []
total_try = []

for decimal in decimals:
    total_try.append(0)
    # calcula o candidato a r
    print("decimal medido: ", decimal)
    numerator, r = fraction(decimal, 2**n)
    print("candidato a r: ", r)

    # verifica se r é par
    if r % 2 != 0 or r == 0:
        print("Falha\n")
        continue
        
    # multord calcula a order de a mod N
    if multord(a,N)==r:    
        success.append(0)    
        guesses = np.gcd(a**int(r/2) + 1, N), np.gcd(a**int(r/2) - 1, N)
        prime_factors = guesses[0], guesses[1]
        print("Fatores primos encontrados: ", prime_factors)

print("Probabilidade de sucesso: ", 100*len(success)/len(total_try), "% ")


decimal medido:  0
candidato a r:  1
Falha

decimal medido:  16
candidato a r:  2
Fatores primos encontrados:  (3, 7)
Probabilidade de sucesso:  50.0 % 


In [12]:
p = prime_factors[0]
q = prime_factors[1]

In [13]:
phi = (p-1)*(q-1)

# Escolha de um valor que satisfaça d*e = 1 + k * phi
k = 2
d = (1 + (k*phi))/e

private_key = (d, p, q)
print(f'A verdadeira chave privada RSA é {private_key}')

A verdadeira chave privada RSA é (5.0, 3, 7)


In [14]:
print("O texto cifrado interceptado foi\n")
print(''.join(str(p) for p in ciphertext))
print("A mensagem secreta encontrada é\n")
print(''.join(str(p) for p in decoder(ciphertext, private_key)))

O texto cifrado interceptado foi

9069057656127761576564680346803392390489125368904891294632665866811336219192335712665876021576562665872769760213571919231336216681266581294639048926658503353923468034680357656127761576566681
A mensagem secreta encontrada é

Socorram-me subi no ônibus em Marrocos
